In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import PIL

In [ ]:
pretrained_model =  tf.keras.applications.EfficientNetB7(
        weights=None,
        include_top=False ,
        input_shape=[150,150, 3]
    )

In [ ]:
INPUT_DIR = "../input/ranzcr-clip-catheter-line-classification/"

train_df = pd.read_csv(INPUT_DIR+"train.csv")
train_df.index = train_df["StudyInstanceUID"]
del train_df["StudyInstanceUID"]

train_annot_df = pd.read_csv(INPUT_DIR+"train_annotations.csv")
train_annot_df.index = train_annot_df["StudyInstanceUID"]
del train_annot_df["StudyInstanceUID"]

In [ ]:
train_df.head()

In [ ]:
train_annot_df.head()

In [ ]:
classes = list(train_df.columns[:-1])
classes_normal= [name for name in classes[:-1] if name.split(" - ")[1] == "Normal"]
classes_abnormal= [name for name in classes[:-1] if name.split(" - ")[1] == "Abnormal"]
classes_borderline = [name for name in classes[:-1] if name.split(" - ")[1] == "Borderline"]
classes_count = train_df[classes].sum(axis = 0)
num_classes = len(classes_count)


print("Number of Classes: {}".format(num_classes))
classes_count

In [ ]:
patient_ids = train_df["PatientID"].unique()
patientwise_count = train_df['PatientID'].value_counts()
num_patients = len(patientwise_count)
print("Number of patients: ",num_patients)
patientwise_count

In [ ]:
train_img_path = INPUT_DIR + "train/"
test_img_path = INPUT_DIR +"test/"
train_files = os.listdir(train_img_path)
test_files = os.listdir(test_img_path)

num_train_files = len(train_files)
num_test_files = len(test_files)

print("Number of training images: {}".format(num_train_files))
print("Number of test images: {}".format(num_test_files))

In [ ]:
labels = []

for name in train_files:
    label = np.array(train_df.loc[name[:-4]][classes].values)
    labels.append(label)
labels = np.array(labels).astype("float32")
print("Train Labels Shape:",labels.shape)

In [ ]:
fig = plt.figure(figsize=(10,10))
height,width=128,128
batch_size=32

for i in range(16):
    img = tf.keras.preprocessing.image.load_img(train_img_path+train_files[i],target_size=(height,width,3))
    arr = tf.keras.preprocessing.image.img_to_array(img)
    ax = plt.subplot(4,4,i+1)
    plt.imshow(arr.astype("uint8"))
    plt.axis("off")
    title  = classes[np.argmax(labels[i])]
    plt.title(title)
plt.show()

In [ ]:
img_arr = []
img_arr_gray = []
for name in train_files:
    img = tf.keras.preprocessing.image.load_img(train_img_path+name,target_size=(height,width,3))
    arr = np.array(tf.keras.preprocessing.image.img_to_array(img))
    #img_arr.append(arr.astype("float32"))
    
    arr_gray = np.array(tf.image.rgb_to_grayscale(arr)).reshape(height,width).astype("float32")
    img_arr_gray.append(arr_gray)
    
#img_arr = np.array(img_arr)
img_arr_gray = np.array(img_arr_gray)

In [ ]:
fig = plt.figure(figsize=(10,10))
img = tf.keras.preprocessing.image.load_img(train_img_path+train_files[0],target_size=(height,width,3))
arr = tf.keras.preprocessing.image.img_to_array(img)
plt.subplot(1,2,1)
plt.imshow(arr.astype("uint8"))
plt.axis("off")
plt.title("ORIGINAL")

arr_gray = np.array(tf.image.rgb_to_grayscale(arr)).reshape(height,width)
plt.subplot(1,2,2)
plt.imshow(arr_gray.astype("uint8"),cmap="gray")
plt.axis("off")
plt.title("GRAY SCALE")
plt.show()

In [ ]:
EPOCHS=100

def create_callbacks(model_save_path,verbose=1):
    
    verbose = int(verbose>0)
    
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)
    
    cpk_path = f'{model_save_path}/model.h5'

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=verbose
    )

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_auc',
        mode='max',
        factor=0.1,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor='val_auc',
        mode='max',
        patience=10, 
        verbose=verbose
    )
    
    callbacks = [checkpoint, reducelr, earlystop]
    
    return callbacks

callbacks = create_callbacks("./",verbose=1)

In [ ]:
from sklearn.metrics import roc_auc_score,f1_score
import tensorflow_addons as tfa

f1 = tfa.metrics.F1Score(num_classes=11)

def roc_metric(labels, predictions):
    return roc_auc_score(labels, predictions)

def f1_metric(labels, predictions):
    return f1_score(labels, predictions)
auc = tf.keras.metrics.AUC(multi_label=True)

def loss_fn(y_true,y_pred):
        return tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true,logits=y_pred)

In [ ]:
from tensorflow.keras.optimizers import SGD

with tf.device('/device:GPU:0'):
    data_augmentation = tf.keras.models.Sequential([
       layers.experimental.preprocessing.RandomFlip("horizontal",input_shape=(height, width,1)),
       layers.experimental.preprocessing.RandomRotation(0.1),
       layers.experimental.preprocessing.RandomZoom(0.1),
       layers.experimental.preprocessing.RandomContrast(0.2),
       layers.experimental.preprocessing.RandomTranslation(0.2,0.2, fill_mode='reflect', interpolation='bilinear'),
       layers.experimental.preprocessing.RandomFlip(mode="horizontal")

    ])
    
    pretrained_model =  tf.keras.applications.EfficientNetB3(
        weights=None,
        include_top=False ,
        input_shape=[height,width,3]
    )
    
    inputs = tf.keras.Input(shape = (height,width,1))
    x = data_augmentation(inputs)
    x = layers.experimental.preprocessing.Rescaling(1/255.)(x)
    x = layers.Conv2D(3,(3,3),padding='same')(x)
    x = pretrained_model(x)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes,activation="sigmoid")(x)
    
    model = tf.keras.Model(inputs,outputs)
    
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(
        optimizer=sgd,
        loss = "binary_crossentropy",
        metrics=[auc]
    )

    
    history=model.fit(
        x = img_arr_gray,
        y = labels,
        validation_split = 0.2,
        shuffle = True,
        batch_size=batch_size,                        
        epochs=EPOCHS,
        verbose=1,
        callbacks = [callbacks]
       )

In [ ]:
model.summary()

In [ ]:
test_img_arr_gray = []
for name in test_files:
    img = tf.keras.preprocessing.image.load_img(test_img_path+name,target_size=(height,width,3))
    arr = np.array(tf.keras.preprocessing.image.img_to_array(img))
    #test_img_arr.append(arr.astype("float32"))
    
    arr_gray = np.array(tf.image.rgb_to_grayscale(arr)).reshape(height,width).astype("float32")
    test_img_arr_gray.append(arr_gray)
    
#test_img_arr = np.array(img_arr)
test_img_arr_gray = np.array(test_img_arr_gray)

In [ ]:
result = model.predict(test_img_arr_gray)
test_index = [name[:-4] for name in test_files]
test_df = pd.DataFrame(result,index=test_index,columns=classes)
test_df.index.name = "StudyInstanceUID"
test_df.head()
test_df.to_csv("./submission.csv")

In [ ]:
os.remove("./model.h5")